In [41]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [42]:
movies_csv = pd.read_csv('Amazon - Movies and TV Ratings.csv')

In [43]:
movies_df = pd.DataFrame(movies_csv)
movies_df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Exploratory Data Analysis

In [44]:
movies_df.drop(columns='user_id', inplace=True)

In [45]:
# Which movies have maximum views/ratings?
count_df = pd.DataFrame(movies_df.count(), columns=['Num_Views_Ratings'])
Top_5_Viewed_Rated = count_df.sort_values(by=['Num_Views_Ratings'], ascending = False).head(n=6)
print(Top_5_Viewed_Rated)

          Num_Views_Ratings
Movie127               2313
Movie140                578
Movie16                 320
Movie103                272
Movie29                 243
Movie91                 128


In [46]:
# What is the average rating for each movie? Define the top 5 movies with the maximum ratings.
mean = movies_df.mean()
mean_df = pd.DataFrame(mean, columns=['Avg_Ratings'])
Top_5_Rated_Movies = mean_df.sort_values(by='Avg_Ratings', ascending=False).head()
print(Top_5_Rated_Movies)

         Avg_Ratings
Movie1           5.0
Movie66          5.0
Movie76          5.0
Movie75          5.0
Movie74          5.0


In [47]:
# Define the top 5 movies with the least audience
Bottom_5_Viewed_Rated = count_df.sort_values(by=['Num_Views_Ratings'], ascending = False).tail()
print(Bottom_5_Viewed_Rated)

          Num_Views_Ratings
Movie54                   1
Movie116                  1
Movie115                  1
Movie55                   1
Movie1                    1


Recommendation Model: 

In [48]:
# Some of the movies hadn’t been watched and therefore, are not rated by the users. Netflix would like to take this as an opportunity and build a machine learning recommendation algorithm which provides the ratings for each of the users.

In [50]:
# Divide the data into training and test data
# pd.set_option('display.max_rows', None)
count_df.reset_index()
mean_df.reset_index()

count_list = list(count_df['Num_Views_Ratings'])
mean_df['Count'] = count_list
merged_df = mean_df.reindex(columns = ['Count', 'Avg_Ratings'])
merged_df.sort_values(by='Count', ascending=False)


,Count,Avg_Ratings
Movie127,2313,4.111976
Movie140,578,4.833910
Movie16,320,4.518750
Movie103,272,4.562500
Movie29,243,4.806584
Movie91,128,4.578125
Movie92,101,4.772277
Movie89,83,4.578313
Movie158,66,4.818182
Movie108,54,4.666667


In [38]:
cutoff = 2
X = np.array(merged_df.loc[merged_df['Count']>2])
y = np.array(merged_df.loc[merged_df['Count']<2])

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [90, 89]

In [25]:
# Build a recommendation model on training data
scaler=MinMaxScaler(feature_range=(0,1))
final_data = merged_df.values
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(final_data)
x_train_data,y_train_data=[],[]
for i in range(60,len(train)):
    x_train_data.append(scaled_data[i-60:i,0])
    y_train_data.append(scaled_data[i,0])

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train)
X_test = scaler.transform

In [29]:
# Make predictions on the test data
lstm_model=Sequential()
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(np.shape(x_train_data)[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))
model_data=merged_df[len(merged_df)-len(test)-60:].values
model_data=model_data.reshape(-1,1)
model_data=scaler.transform(model_data)

ValueError: X has 1 features, but MinMaxScaler is expecting 2 features as input.